# Search Engine with Tools & Agents

## Creating Tools

### 1. In-built Tools

In [10]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [11]:
# using the inbuilt tool of wikipedia
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=300)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki_tool.name

'wikipedia'

In [12]:
api_wrapper_arx = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=300)
arx_tool = ArxivQueryRun(api_wrapper=api_wrapper_arx)
arx_tool.name

'arxiv'

In [13]:
tools = [wiki_tool, arx_tool]

### 2. Custom Tools

In [14]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

In [16]:
loader = WebBaseLoader(web_path="https://ts.llamaindex.ai/modules/llamacloud")
documents = loader.load()
doc_chunks = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(documents)
doc_chunks

[Document(metadata={'source': 'https://ts.llamaindex.ai/modules/llamacloud', 'title': 'LlamaCloud | LlamaIndex.TS', 'description': 'LlamaCloud is a new generation of managed parsing, ingestion, and retrieval services, designed to bring production-grade context-augmentation to your LLM and RAG applications.', 'language': 'en'}, page_content='LlamaCloud | LlamaIndex.TS\n\n\n\n\n\n\n\n\n\n\nSkip to main contentLlamaIndex.TSDocsEnglishEnglishBlogGitHubSearchWhat is LlamaIndex?Getting StartedGuidesExamplesRecipesModulesDocument / NodesPromptsLoadersVector StoresIngestion PipelineQuery EnginesAgentsEmbeddingsEvaluatingLLMsNode PostprocessorsChatEngineIndexNodeParserRetrieverResponseSynthesizerStorageLlamaCloudObservabilityllamaindexModulesLlamaCloudOn this pageLlamaCloud\nLlamaCloud is a new generation of managed parsing, ingestion, and retrieval services, designed to bring production-grade context-augmentation to your LLM and RAG applications.\nCurrently, LlamaCloud supports\n\nManaged Inge

In [18]:
vectorstore = FAISS.from_documents(doc_chunks, HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"))
retriever = vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000024F609EFC10>)

In [44]:
# turning the retriever into tool
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever=retriever, name="Llamacloud-search",
                                       description="Search any information about llamacloud")
retriever_tool.name

'Llamacloud-search'

In [45]:
# combinig all tools together
tools = [wiki_tool, arx_tool, retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Projects and Learning\\Langchain\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=300)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=300)),
 Tool(name='Llamacloud-search', description='Search any information about llamacloud', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000024F413D3060>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000024F609EFC10>

## Running all tools with Agents and LLMs

In [46]:
## LLM
from langchain_groq import ChatGroq
import os, dotenv
dotenv.load_dotenv()

api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(model="llama-3.1-70b-versatile", api_key=api_key)

In [47]:
# prompt template
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [48]:
# Agents
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm=llm, tools=tools, prompt=prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_has

In [49]:
# Agent Executer
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwc

In [51]:
# invoking the agent executor
agent_executor.invoke({"input": "What is the Llamacloud?"})



> Entering new AgentExecutor chain...

Invoking: `Llamacloud-search` with `{'query': 'Llamacloud'}`


LlamaCloudIndex
LlamaCloudRetriever
PreviousStorageNextObservabilityAccessCreate a Managed IndexUse a Managed IndexAPI ReferenceDocsAPICommunityDiscordTwitterMoreGitHubCopyright © 2024 LlamaIndex. Built with Docusaurus.

LlamaCloud | LlamaIndex.TS










Skip to main contentLlamaIndex.TSDocsEnglishEnglishBlogGitHubSearchWhat is LlamaIndex?Getting StartedGuidesExamplesRecipesModulesDocument / NodesPromptsLoadersVector StoresIngestion PipelineQuery EnginesAgentsEmbeddingsEvaluatingLLMsNode PostprocessorsChatEngineIndexNodeParserRetrieverResponseSynthesizerStorageLlamaCloudObservabilityllamaindexModulesLlamaCloudOn this pageLlamaCloud
LlamaCloud is a new generation of managed parsing, ingestion, and retrieval services, designed to bring production-grade context-augmentation to your LLM and RAG applications.
Currently, LlamaCloud supports

Managed Ingestion API, handling parsing and d

{'input': 'What is the Llamacloud?',
 'output': 'LlamaCloud is a managed parsing, ingestion, and retrieval service designed to bring production-grade context-augmentation to LLM and RAG applications. It currently supports managed ingestion and retrieval APIs, and is available as a private beta to a limited set of enterprise partners.'}

In [52]:
agent_executor.invoke({"input": "What is Generative AI?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Generative AI'}`


Page: Generative artificial intelligence
Summary: Generative artificial intelligence (generative AI, GenAI, or GAI) is artificial intelligence capable of generating text, images, videos, or other data using generative models, often in response to prompts. Generative AI models learn the patterns and Based on the tool output, here is the final answer:

Generative artificial intelligence (generative AI, GenAI, or GAI) is artificial intelligence capable of generating text, images, videos, or other data using generative models, often in response to prompts. Generative AI models learn the patterns and relationships in large datasets and can generate new, synthetic data that resembles the original data.

> Finished chain.


{'input': 'What is Generative AI?',
 'output': 'Based on the tool output, here is the final answer:\n\nGenerative artificial intelligence (generative AI, GenAI, or GAI) is artificial intelligence capable of generating text, images, videos, or other data using generative models, often in response to prompts. Generative AI models learn the patterns and relationships in large datasets and can generate new, synthetic data that resembles the original data.'}

In [54]:
agent_executor.invoke({"input": "Tell me what's this paper 1706.03762 all about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks The paper "Attention Is All You Need" by Ashish Vaswani et al. presents a novel approach to sequence transduction models, which are commonly used in natural language processing tasks. The authors propose a new architecture that relies solely on self-attention mechanisms, eliminating the need for recurrent or convolutional neural networks. This approach is claimed to be more efficient and effective than traditional models.

> Finished chain.


{'input': "Tell me what's this paper 1706.03762 all about?",
 'output': 'The paper "Attention Is All You Need" by Ashish Vaswani et al. presents a novel approach to sequence transduction models, which are commonly used in natural language processing tasks. The authors propose a new architecture that relies solely on self-attention mechanisms, eliminating the need for recurrent or convolutional neural networks. This approach is claimed to be more efficient and effective than traditional models.'}